# Install Yolo

## Clone Darknet

In [1]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15386, done.
remote: Total 15386 (delta 0), reused 0 (delta 0), pack-reused 15386
Receiving objects: 100% (15386/15386), 13.99 MiB | 18.72 MiB/s, done.
Resolving deltas: 100% (10347/10347), done.


## Compile

In [2]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

/content/darknet


In [3]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [5]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

chmod +x *.sh
nvcc -gencode arch=compute_35,code=sm_35 -gencode arch=compute_50,code=[sm_50,compute_50] -gencode arch=compute_52,code=[sm_52,compute_52] -gencode arch=compute_61,code=[sm_61,compute_61] -gencode arch=compute_70,code=[sm_70,compute_70] -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF --compiler-options "-Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC" -c ./src/dropout_layer_kernels.cu -o obj/dropout_layer_kernels.o
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc -gencode arch=compute_35,code=sm_35 -gencode arch=compute_50,code=[sm_50,compute_50] -gencode arch=compute_52,code=[sm_52,compute_5

# Library

In [3]:

# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline
from darknet import *

# Predict function

In [4]:
# darknet helper function to run detection on image
def darknet_helper(img, width, height):
  darknet_image = make_image(width, height, 3)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_resized = cv2.resize(img_rgb, (width, height),
                              interpolation=cv2.INTER_LINEAR)

  # get image ratios to convert bounding boxes to proper size
  img_height, img_width, _ = img.shape
  width_ratio = img_width/width
  height_ratio = img_height/height

  # run model on darknet style image to get detections
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)
  return detections, width_ratio, height_ratio

In [5]:
def yolo_predict(input_file):
  img = cv2.imread(input_file)
  detections, width_ratio, height_ratio = darknet_helper(img, width, height)
  for label, confidence, bbox in detections:
      left, top, right, bottom = bbox2points(bbox)
      left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
      cv2.rectangle(img, (left, top), (right, bottom), class_colors[label], 2)
      cv2.putText(img, "{} [{:.2f}]".format(label, float(confidence)),
                        (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        class_colors[label], 2)
  cv2.imwrite(input_file, img)

# Web interface

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!pip install flask-ngrok

In [11]:
!pip install pyngrok
!ngrok authtoken 23594Kn4vbYxJ0zcGK5G0J3Gqux_XuEKpCypMYoW1nA8hfG9

     |████████████████████████████████| 745 kB 5.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=386fd99e0aca83b98b01b69cc29ab5acd9f1d6392c821162578f165968d109a6
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


## Load network

In [6]:
network, class_names, class_colors = load_network("/content/drive/MyDrive/Yolo/yolo_testing.cfg", 
                                                  "/content/drive/MyDrive/Yolo/obj.data", "/content/drive/MyDrive/Yolo/yolov3_training_last.weights")
width = network_width(network)
height = network_height(network)

In [7]:
import os
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok
app = Flask(__name__,template_folder='/content/drive/MyDrive/Yolo')
run_with_ngrok(app)   
app.config["IMAGE_UPLOADS"] = "/content/drive/MyDrive/Yolo"


@app.route("/")
def home():
    return render_template('index.html')
# Route to upload image

@app.route('/upload-image', methods=['GET', 'POST'])
def upload_image():
    if request.method == "POST":
        if request.files:
            image = request.files["image"]
            save_path = os.path.join(app.config["IMAGE_UPLOADS"], image.filename)
            image.save(save_path)
            yolo_predict(save_path)
            return render_template("index.html", uploaded_image=image.filename)
    return render_template("index.html")

@app.route('/uploads/<filename>')
def send_uploaded_file(filename=''):
    from flask import send_from_directory
    return send_from_directory(app.config["IMAGE_UPLOADS"], filename)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8cf2-35-194-4-142.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [05/Jan/2022 03:18:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 03:18:39] "GET /uploads/ HTTP/1.1" 404 -
127.0.0.1 - - [05/Jan/2022 03:18:40] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [05/Jan/2022 03:18:55] "POST /upload-image HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 03:18:55] "GET /uploads/road3.png HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 03:19:05] "POST /upload-image HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 03:19:06] "GET /uploads/road11.png HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 03:36:58] "POST /upload-image HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 03:36:58] "GET /uploads/road78.png HTTP/1.1" 200 -
[2022-01-05 03:40:05,509] ERROR in app: Exception on /upload-image [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self